In [ ]:
import numpy as np
import cv2 
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import glob
import os
import re
import imutils
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from scipy import ndimage
import collections
from skimage.draw import circle, line
from skimage.measure import regionprops
import tqdm
import matplotlib.image as mpimg
from scipy.spatial import distance as dist
import tqdm

In [ ]:
def return_cord(i_0):    
    cx = []
    cy = []

    shifted = cv2.pyrMeanShiftFiltering(i_0, 10, 50)
    gray = cv2.cvtColor(shifted,cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    D = ndimage.distance_transform_edt(thresh)
    localMax = peak_local_max(D, indices=False, min_distance=9, labels=thresh)
    markers = ndimage.label(localMax, structure=np.ones((3, 3)))[0]
    labels = watershed(-D, markers, mask=thresh)



    for label in np.unique(labels):
        if label == 0:
            continue




        mask = np.zeros(gray.shape, dtype='uint8')
        mask[labels == label] = 255
        cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        cnts = imutils.grab_contours(cnts)
        c = max(cnts, key=cv2.contourArea)
        ((x, y), r) = cv2.minEnclosingCircle(c)



        cx.append(int(x))
        cy.append(int(y))
        
        l1 = np.column_stack((cx,cy))
        

    return l1, cx, cy

# First Image t0 to t1

In [ ]:
def eucl_dist_matrix_t1(cor_t, cor_t1):
    
    d = dist.cdist(cor_t, cor_t1)
    rows = d.min(axis=1).argsort()
    cols = d.argmin(axis=1)[rows]
    
    list_id_index = (list(zip(rows, cols)))
    list_id_index = sorted(list_id_index)
    
    
    
    
    return rows, cols, list_id_index ,d 

In [ ]:
def find_duplicates_col_t1(rows, columns,list_id_index, distance_matrix):
    
    col = columns.tolist()
    duplicates = []
    for item in col:
        if col.count(item) > 1:
            duplicates.append(item)
    duplicate_columns = set(duplicates)
    
    d = []
    r = []
    c = []
    for ix, (a, b) in enumerate(zip(rows, columns)):
        for i in duplicate_columns: 
            if b == i:
                d.append(distance_matrix[a][b])
                r. append(a)
                c.append(b)
                
    w = (list(zip(r,c,d)))
    w.sort(key = lambda t: t[1])
    w
    
    temp = {}
    for row, col, di in w:
        if col not in temp: # we see this key for the first time
            temp[col] = (row, col, di)
        else:
            if temp[col][2] > di: # the new date is larger than the old one
                temp[col] = (row, col, di)

    result = temp.values()

    result = list(result)

    l2set = set(result)
    colums_to_remove = [x for x in w if x not in l2set] # cordinates that have to be removed
     
    rem_row, rem_col, rem_dit = zip(*colums_to_remove)
    rem_cor = (list(zip(rem_row,rem_col)))
    
    rem_cor = set(rem_cor)
    
    keep_cor = [x for x in list_id_index if x not in rem_cor]

    keep_cor.sort(key = lambda t: t[1])
    
    id_cell, index = zip(*keep_cor)

    
    return index, duplicate_columns

In [ ]:
def remove_duplicates_t1(x, y, index, id_cell_pr):
    
    large = max(index)
    lst = list(range(0,(large+1)))
    
    ix = set(index)
    
    missing_ix = [x for x in lst if x not in ix]
    
    centroid_t = (list(zip(x,y, id_cell_pr)))
    
    real_centroid = [i for j, i in enumerate(centroid_t) if j not in missing_ix]
    
    x_i, y_i, id_cell_new = zip(*real_centroid)
    
    stacket_centroids = np.column_stack((x_i,y_i))
    
    
    
    return x_i, y_i, stacket_centroids, id_cell_new

In [ ]:
def get_id_prime(x, y):
    
    df = pd.DataFrame(list(zip(x, y)), 
               columns =['row', 'col']) 

    id_cell_pr = []
    x = []
    y = []

    for i in range(len(df)):
        x.append(df.iloc[i,0])
        y.append(df.iloc[i,1])
        



    for idx, i in enumerate(x):
        
        id_cell_pr.append(idx)
    
    return id_cell_pr

In [ ]:
# def get_image(x, y, id_cell, image, number_image):
    
#     df2 = pd.DataFrame(list(zip(x, y, id_cell)), 
#                columns =['row', 'col', 'id_cell']) 


#     x = []
#     y = []
#     corrdinate = []
#     for i in range(len(df2)):
#         x.append(df2.iloc[i,0])
#         y.append(df2.iloc[i,1])
#         corrdinate.append(df2.iloc[i,2])



#     for idx, i in enumerate(x):



#         cv2.circle(image, (x[idx], y[idx]), 3, (0, 255, 255), -1)
#         cv2.putText(image, "{0}".format(corrdinate[idx]), (x[idx] - 20, y[idx] - 20),
#             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

#     cv2.imwrite('img_try_fina_track_{0}.png'.format(number_image), image)

# From t1 to rest of data set



In [ ]:
# IF Index duplicates cols > 0

def remove_dup_cols(row1, col1, dup_set2, x0, y0, id_cell,d, list_id_index1):

    dis = []
    r = []
    c = []

    for ix, (a, b) in enumerate(zip(row1, col1)):
        for i in dup_set2: 
            if b == i:
                dis.append(d[a][b])
                r. append(a)
                c.append(b)
                    
    w = (list(zip(r,c,dis)))
    w.sort(key = lambda t: t[1])
        
    temp = {}
    for row, col, dis in w:
        if col not in temp: # we see this key for the first time
            temp[col] = (row, col, dis)
        else:
            if temp[col][2] > dis: 
                temp[col] = (row, col, dis)

    result = temp.values()

    result = list(result)

        
    l2set = set(result)
    l3 = [x for x in w if x not in l2set]
        
    rem_row, rem_col, rem_dit = zip(*l3)
    rem_cor = (list(zip(rem_row,rem_col)))
        
    rem_cor = set(rem_cor)
    keep_cor = [x for x in list_id_index1 if x not in rem_cor]     # cordinates that have to be removed
    keep_cor.sort(key = lambda t: t[1])
        
        
    new_rowt1, new_colt1 = zip(*keep_cor)
    large = len(new_rowt1)
    large
    lst = list(range(0,(large+1)))
    ix = set(new_rowt1)

    m = [x for x in lst if x not in ix]

    centroid_t0 = (list(zip(x0,y0,id_cell)))

    real_centroid0 = [i for j, i in enumerate(centroid_t0) if j not in m]

    x_0, y_0, id_cell0= zip(*real_centroid0)

    correct_0 = np.column_stack((x_0,y_0)) 
        
    return correct_0, id_cell0

In [ ]:
# IF Index unused cols > 0

def remove_unused_cols (idx_unused_col, x1, y1):
    
    miss_idx = idx_unused_col
    miss_idx
    centroid_t1 = (list(zip(x1, y1)))
    real_centroid1 = [i for j, i in enumerate(centroid_t1) if j not in miss_idx]
    
    return real_centroid1
        


In [ ]:
def euc_matrix_2(data_t1, data_t2):
    
    rows1, cols1, matrix1, d1 = eucl_dist_matrix_t1(data_t1, data_t2 )
    row1, col1 = zip(*matrix1)

    correct_1= []

    for ix, (i,j) in enumerate(zip(row1, col1)):
        correct_1.insert(ix, data_t2[j])
            
    x_1, y_1 = zip(*correct_1)
    
    return x_1, y_1

In [1]:
def final_fun(dup_set2, idx_unused_col, d1, id_cell, list_id_index1, cor_now,
             new_cor_before):
    
    row1, col1         = zip(*list_id_index1)
    x_before, y_before = zip(*new_cor_before)
    x_now, y_now       = zip(*cor_now)

    if len(dup_set2) > 0 and len(idx_unused_col) > 0:

        correct_0, id_cell_new = remove_dup_cols(row1, col1, dup_set2, x_before, y_before, id_cell,d1,list_id_index1)

        real_centroid1      = remove_unused_cols (idx_unused_col, x_now, y_now)

        xt1, yt1 = euc_matrix_2(correct_0 ,real_centroid1)

        #get_image(xt1, yt1, id_cell_new, img_now, num_img)
        
        return list(zip(xt1, yt1)), id_cell_new



    elif len(dup_set2) > 0 and len(idx_unused_col) == 0:

        correct_0, id_cell_new = remove_dup_cols(row1, col1, dup_set2, x_before, y_before, id_cell, list_id_index1)

        xt1, yt1 = euc_matrix_2(correct_0 , cor_now)

        #get_image(xt1, yt1, id_cell0, img_now, num_img)
        
        return list(zip(xt1, yt1)), id_cell_new


    elif len(dup_set2)  == 0 and len(idx_unused_col) > 0:

        real_centroid1      = remove_unused_cols (idx_unused_col, x_now, y_now)

        xt1, yt1 = euc_matrix_2(new_cor_before , real_centroid1)

        #get_image(xt1, yt1, id_cell, img_now, num_img)
        
        return list(zip(xt1, yt1)), id_cell


    else:

        xt1, yt1 = euc_matrix_2(new_cor_before , cor_now)

        #get_image(xt1, yt1, id_cell, img_now, num_img)
        
        return list(zip(xt1, yt1)), id_cell
